In [1]:
# import packages
import json, re, nltk
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.stem.wordnet import WordNetLemmatizer
from tmtoolkit.topicmod.evaluate import metric_coherence_gensim

import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [2]:
df = pd.read_csv('yelp.csv')
df.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,J4a2TuhDasjn2k3wWtHZnQ,RNm_RWkcd02Li2mKPRe7Eg,xGXzsc-hzam-VArK6eTvtw,1.0,2,0,0,"This place used to be a cool, chill place. Now...",2018-01-21 04:41:03
1,FdoBFTjXXMn4hVnJ59EtiQ,eLAYHxHUutiXswy-CfeiUw,WQFn1A7-UAA4JT5YWiop_w,1.0,0,0,0,"They NEVER seem to get our \norder correct, se...",2017-09-08 23:26:10
2,m1GlqFGIN5eayrbb2IbRZg,B7YSV6r1ePAXc69FkDDuZw,wZgUAuDuEGPEzKK-PsngKQ,1.0,0,0,0,I wish I could give them zero stars. The call ...,2014-06-27 22:06:55
3,ucFOnqgaV40oQ2YNyz5ddQ,JHXQEayrDHOWGexs0dCviA,KXCXaF5qimmtKKqnPc_LQA,1.0,0,0,0,Great coffee and pastries. Baristas are excell...,2018-03-03 23:45:25
4,-QpNdU_p44GR0NcRxDRyNQ,ffJp-ZN80M4sSkDL8Ra18w,WDGeeyeK7bG0cvq_ZglAdA,1.0,0,0,0,Almost desolate restaurant and dingy evironmen...,2009-03-01 01:05:50


In [3]:
# initial stopwords
stopwords = nltk.corpus.stopwords.words('english')
# more stopwords
extra_stopwords = ['nt', 'good', 'bad', 'tell', 'say', 'ask', 'use', 'come', 'make', 
                   'company', 'stay', 'want', 'know', 'like', 'anymore', 'lot', 'great', 
                   'ok', 'nice', 'dish', 'really', 'look', 'better', 'guy', 'dr', 'try',
                   'let', 'need', 'alright', 'place', 'food', 'buy', 'eat']

# function that handles tokenization
def text_process(text):
    clean_tokens = []
    lmtzr = WordNetLemmatizer()
    
    # filter stopwords and change format
    for token in nltk.word_tokenize(text):
        token = token.replace("'s", " ").replace("n’t", " not").replace("’ve", " have")
        token = re.sub(r'[^a-zA-Z0-9 ]', '', token)
        if token.lower() not in stopwords and token.isalpha():
            clean_tokens.append(token.lower())
    
    result = [lmtzr.lemmatize(token, 'v') for token in clean_tokens]
    filtered_token = []
    
    # second filter
    for token in result:
        if token.lower() not in extra_stopwords: 
            filtered_token.append(token)
    return filtered_token

# construct tfidf matrix
def build_tfidf(data, text_process, max_df, min_df):
    # initialize
    tfidf_model = TfidfVectorizer(max_df=max_df, 
                                  min_df=min_df, 
                                  stop_words='english',
                                  use_idf=True, 
                                  tokenizer=text_process)

    tfidf_matrix = tfidf_model.fit_transform(data) #fit the vectorizer to synopses
    
    print ("In total, there are " + str(tfidf_matrix.shape[0]) + \
           " reviews and " + str(tfidf_matrix.shape[1]) + " terms.")
    return tfidf_matrix, tfidf_model

# build lda 
def conduct_lda(tfidf_matrix, data, num_components):
    lda = LatentDirichletAllocation(n_components=num_components)
    lda_output = lda.fit_transform(tfidf_matrix)
    # check shape
    print(lda_output.shape)
    
    return lda_output, lda

# print top n keywords for each topic
def generate_topic_words(tfidf_model, lda_model, n_words):
    words = np.array(tfidf_model.get_feature_names())
    topic_words = []
    # for each topic, we have words weight
    for topic_words_weights in lda_model.components_:
        top_words = topic_words_weights.argsort()[::-1][:n_words]
        topic_words.append(words.take(top_words))
    # store keywords in a dataframe
    df_topic_words = pd.DataFrame(topic_words)
    df_topic_words.columns = ['Word '+ str(i) for i in range(df_topic_words.shape[1])]
    df_topic_words.index = ['Topic '+ str(i) for i in range(df_topic_words.shape[0])]
    return df_topic_words

# pipeline for whole process
def pipeline(star_level, max_df, min_df, num_components, n_words):
    # select data
    data = df[df['stars']==star_level]['text'].tolist()
    # train tfidf matrix
    tfidf_matrix, tfidf_model = build_tfidf(data, text_process, max_df, min_df)
    # train lda model
    lda_output, lda_model = conduct_lda(tfidf_matrix, data, num_components)
    # generate top keywords
    keywords = generate_topic_words(tfidf_model, lda_model, n_words)
    
    return keywords, lda_model, tfidf_matrix, tfidf_model

In [4]:
# star level 1
df1, lda_model, tfidf_matrix, tfidf_model = pipeline(1, 0.99, 0.01, 3, 20)

In total, there are 8000 reviews and 738 terms.
(8000, 3)


In [5]:
pyLDAvis.sklearn.prepare(lda_model, tfidf_matrix, tfidf_model)

/opt/anaconda3/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.113868  0.075708       1        1  43.227009
2     -0.050609 -0.097974       2        1  31.668804
1      0.164477  0.022266       3        1  25.104186, topic_info=           Term        Freq       Total Category  logprob  loglift
645       taste  147.000000  147.000000  Default  30.0000  30.0000
113     chicken  139.000000  139.000000  Default  29.0000  29.0000
430       order  369.000000  369.000000  Default  28.0000  28.0000
643       table  154.000000  154.000000  Default  27.0000  27.0000
456       pizza  106.000000  106.000000  Default  26.0000  26.0000
..          ...         ...         ...      ...      ...      ...
599       small   49.267264   75.946488   Topic3  -5.1278   0.9494
66         bite   48.052124   71.092806   Topic3  -5.1528   0.9904
535  restaurant   55.017195  157.115620   Topic3  -5.0174   0.3328
657        time   60.645374  322.236704   Topic3  -4.9200  -0.2881
576       serve   47.567729   92.358853   Topic3  -5.1629   0.7186

[200 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
5         1  0.961673      account
6         1  0.127474  acknowledge
6         2  0.849829  acknowledge
12        1  0.970194      address
23        1  0.944863       answer
...     ...       ...          ...
720       2  0.226080         work
720       3  0.020553         work
724       1  0.284184        worst
724       2  0.429434        worst
724       3  0.290499        worst

[330 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2])

In [6]:
# star level 2
df2, lda_model, tfidf_matrix, tfidf_model = pipeline(2, 0.99, 0.01, 3, 20)

In total, there are 8000 reviews and 768 terms.
(8000, 3)


In [7]:
pyLDAvis.sklearn.prepare(lda_model, tfidf_matrix, tfidf_model)

/opt/anaconda3/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.156365 -0.022428       1        1  36.306822
0      0.113194 -0.063910       2        1  32.741157
2      0.043170  0.086338       3        1  30.952021, topic_info=           Term        Freq       Total Category  logprob  loglift
118     chicken  169.000000  169.000000  Default  30.0000  30.0000
669       table  169.000000  169.000000  Default  29.0000  29.0000
726        wait  215.000000  215.000000  Default  28.0000  28.0000
207       drink  201.000000  201.000000  Default  27.0000  27.0000
674       taste  176.000000  176.000000  Default  26.0000  26.0000
..          ...         ...         ...      ...      ...      ...
233  experience   59.521733  147.129925   Topic3  -5.1930   0.2678
639       staff   59.487651  149.926476   Topic3  -5.1935   0.2484
736       water   51.140764   87.991007   Topic3  -5.3447   0.6301
407        menu   53.008451  126.875811   Topic3  -5.3088   0.3000
192      dinner   50.078568   85.829062   Topic3  -5.3657   0.6340

[199 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
22        2  0.967652  appointment
26        1  0.071553       arrive
26        2  0.228968       arrive
26        3  0.701216       arrive
35        1  0.975776    authentic
...     ...       ...          ...
745       1  0.097115         wine
745       3  0.912885         wine
753       1  0.053509         work
753       2  0.733839         work
753       3  0.214036         work

[347 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 3])

In [8]:
# star level 3
df3, lda_model, tfidf_matrix, tfidf_model = pipeline(3, 0.99, 0.01, 5, 20)

In total, there are 8000 reviews and 765 terms.
(8000, 5)


In [9]:
pyLDAvis.sklearn.prepare(lda_model, tfidf_matrix, tfidf_model)

/opt/anaconda3/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.122133  0.040841       1        1  30.623508
4      0.012403  0.029930       2        1  28.132631
3     -0.076749 -0.012245       3        1  19.952834
2     -0.076932  0.053900       4        1  12.532497
0      0.019145 -0.112426       5        1   8.758530, topic_info=         Term        Freq       Total Category  logprob  loglift
483     pizza  155.000000  155.000000  Default  30.0000  30.0000
133    coffee   96.000000   96.000000  Default  29.0000  29.0000
449     order  297.000000  297.000000  Default  28.0000  28.0000
661     sushi   89.000000   89.000000  Default  27.0000  27.0000
187  delivery   46.000000   46.000000  Default  26.0000  26.0000
..        ...         ...         ...      ...      ...      ...
637      star   17.560827  151.987196   Topic5  -5.1356   0.2770
680     think   17.746265  181.157313   Topic5  -5.1251   0.1120
668     taste   17.295137  160.320829   Topic5  -5.1508   0.2084
371      long   15.543224  102.189346   Topic5  -5.2576   0.5519
269  friendly   15.611553  130.417812   Topic5  -5.2532   0.3124

[295 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
18        2  0.758115   appetizer
18        3  0.216604   appetizer
19        2  0.639677  appetizers
19        3  0.336672  appetizers
21        1  0.485097        area
...     ...       ...         ...
749       5  0.083466        work
759       1  0.766555        year
759       2  0.085173        year
759       3  0.085173        year
759       4  0.085173        year

[835 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 5, 4, 3, 1])

In [10]:
# star level 4
df4, lda_model, tfidf_matrix, tfidf_model = pipeline(4, 0.99, 0.01, 7, 20)

In total, there are 8000 reviews and 690 terms.
(8000, 7)


In [11]:
pyLDAvis.sklearn.prepare(lda_model, tfidf_matrix, tfidf_model)

/opt/anaconda3/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.079244  0.016615       1        1  32.175678
5      0.023517 -0.017040       2        1  18.544947
4      0.050308 -0.021641       3        1  13.913306
0      0.065434 -0.080068       4        1  11.232746
3      0.046815 -0.020345       5        1   9.275163
6     -0.155573 -0.014818       6        1   8.005875
1      0.048743  0.137298       7        1   6.852285, topic_info=          Term        Freq       Total Category  logprob  loglift
593      sushi  103.000000  103.000000  Default  30.0000  30.0000
428      pizza  129.000000  129.000000  Default  29.0000  29.0000
526    service  289.000000  289.000000  Default  28.0000  28.0000
606       thai   67.000000   67.000000  Default  27.0000  27.0000
597      tacos   63.000000   63.000000  Default  26.0000  26.0000
..         ...         ...         ...      ...      ...      ...
326     little   18.064448  193.438477   Topic7  -4.8193   0.3096
398      order   17.948833  253.216663   Topic7  -4.8257   0.0339
450      price   16.363101  203.852014   Topic7  -4.9182   0.1582
475  recommend   15.657123  137.391957   Topic7  -4.9623   0.5087
500      salad   14.777171  110.714012   Topic7  -5.0202   0.6667

[394 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
2         1  0.392866  accommodate
2         2  0.087304  accommodate
2         3  0.043652  accommodate
2         4  0.043652  accommodate
2         5  0.043652  accommodate
...     ...       ...          ...
681       3  0.424000         wrap
681       4  0.060571         wrap
681       5  0.151428         wrap
681       6  0.030286         wrap
681       7  0.030286         wrap

[1513 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 6, 5, 1, 4, 7, 2])

In [12]:
# star level 5
df5, lda_model, tfidf_matrix, tfidf_model = pipeline(5, 0.99, 0.01, 3, 20)

In total, there are 8000 reviews and 590 terms.
(8000, 3)


In [13]:
pyLDAvis.sklearn.prepare(lda_model, tfidf_matrix, tfidf_model)

/opt/anaconda3/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.118039  0.091799       1        1  38.550969
0     -0.165088  0.030720       2        1  34.607498
2      0.047049 -0.122519       3        1  26.841533, topic_info=          Term        Freq       Total Category  logprob  loglift
93      coffee  116.000000  116.000000  Default  30.0000  30.0000
82     chicken  133.000000  133.000000  Default  29.0000  29.0000
576       work  170.000000  170.000000  Default  28.0000  28.0000
204        fry  111.000000  111.000000  Default  27.0000  27.0000
371      pizza  121.000000  121.000000  Default  26.0000  26.0000
..         ...         ...         ...      ...      ...      ...
284       live   46.794606   91.268683   Topic3  -5.1277   0.6472
283     little   49.219729  130.250161   Topic3  -5.0772   0.3421
131  delicious   51.382637  212.632727   Topic3  -5.0342  -0.1050
504      super   48.081225  115.353592   Topic3  -5.1006   0.4401
179   favorite   47.677613  124.927988   Topic3  -5.1090   0.3519

[205 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
10        1  0.463184        amaze
10        2  0.268812        amaze
10        3  0.268812        amaze
12        2  0.976758       answer
15        2  0.978523  appointment
...     ...       ...          ...
579       2  0.185856        worth
579       3  0.195638        worth
585       1  0.126434        years
585       2  0.865588        years
585       3  0.009726        years

[314 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 3])